In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
df=pd.read_csv(r'final.csv')

In [128]:
df.FLAG_OWN_CAR

0         0
1         0
2         1
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    0
307510    0
Name: FLAG_OWN_CAR, Length: 307511, dtype: int64

In [3]:
df.drop(columns=df.columns[0], axis=1, inplace=True)


In [4]:
df.shape

(307511, 98)

In [39]:
features=['EXT_SOURCE_3',
 'FLAG_CONT_MOBILE',
 'NAME_CONTRACT_TYPE',
 'EXT_SOURCE_2',
 'NAME_EDUCATION_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_16',
 'REGION_RATING_CLIENT_W_CITY',
 'FLAG_DOCUMENT_3',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'NAME_INCOME_TYPE',
 'AMT_ANNUITY',
 'NAME_FAMILY_STATUS',
 'OCCUPATION_TYPE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE','TARGET']

In [40]:
less_df=df[features]

In [41]:
less_df.head()

,EXT_SOURCE_3,FLAG_CONT_MOBILE,NAME_CONTRACT_TYPE,EXT_SOURCE_2,NAME_EDUCATION_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_DOCUMENT_18,FLAG_DOCUMENT_16,REGION_RATING_CLIENT_W_CITY,FLAG_DOCUMENT_3,DEF_60_CNT_SOCIAL_CIRCLE,NAME_INCOME_TYPE,AMT_ANNUITY,NAME_FAMILY_STATUS,OCCUPATION_TYPE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,TARGET
0,0.139376,1.0,Cash loans,0.262949,Secondary / secondary special,M,N,0.0,0.0,2.0,1.0,2.0,Working,24700.5,Single / not married,Laborers,block of flats,"Stone, brick",No,1.0
1,0.529522,1.0,Cash loans,0.622246,Higher education,F,N,0.0,0.0,1.0,1.0,0.0,State servant,35698.5,Married,Core staff,block of flats,Block,No,0.0
2,0.729567,1.0,Revolving loans,0.555912,Secondary / secondary special,M,Y,0.0,0.0,2.0,0.0,0.0,Working,6750.0,Single / not married,Laborers,block of flats,Panel,No,0.0
3,0.558707,1.0,Cash loans,0.650442,Secondary / secondary special,F,N,0.0,0.0,2.0,1.0,0.0,Working,29686.5,Civil marriage,Laborers,block of flats,Panel,No,0.0
4,0.566188,1.0,Cash loans,0.322738,Secondary / secondary special,M,N,0.0,0.0,2.0,0.0,0.0,Working,21865.5,Single / not married,Core staff,block of flats,Panel,No,0.0


In [31]:
from sklearn.preprocessing import LabelEncoder


In [32]:
cat_vals=df.select_dtypes('object').nunique().index.tolist()


In [33]:
for i in cat_vals:
    enc=LabelEncoder()
    df[i]=enc.fit_transform(df[i])

In [65]:
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,100002.0,1.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,7,4,3,1,8,6,5,0,5,0
1,100003.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,4,1,1,1,3,1,39,0,0,0
2,100004.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,7,4,3,1,8,1,11,0,4,0
3,100006.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,7,4,0,1,8,6,5,0,4,0
4,100007.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,7,4,3,1,3,4,37,0,4,0


In [14]:
def engineer_main_features(X):
    
    # Create aggregated features. Idea to do this taken 
    # from neptune-ml's solution at: 
    # https://github.com/neptune-ml/open-solution-home-credit/blob/solution-5/notebooks/eda-application.ipynb
    
    aggregation_recipes = [
        (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                                  ('AMT_CREDIT', 'max'),
                                                  ('EXT_SOURCE_2', 'mean'),
                                                 ]),
        (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                                ('AMT_INCOME_TOTAL', 'mean'),
                                                ('DAYS_REGISTRATION', 'mean')]),
        (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                     ('CNT_CHILDREN', 'mean'),
                                                     ('DAYS_ID_PUBLISH', 'mean')]),
        (['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [
                                                                                               ('EXT_SOURCE_2', 'mean')]),
        (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
                                                      ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
                                                      ('APARTMENTS_AVG', 'mean'),
                                                    
                                                      ('EXT_SOURCE_2', 'mean'),
                                                      ('EXT_SOURCE_3', 'mean'),
                                                      ('NONLIVINGAREA_AVG', 'mean'),
                                                  ]),
        (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean'),
                                                                                ]),
        (['OCCUPATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                               ('CNT_CHILDREN', 'mean'),
                               ('CNT_FAM_MEMBERS', 'mean'),
                               ('DAYS_BIRTH', 'mean'),
                               ('DAYS_EMPLOYED', 'mean'),
                               ('DAYS_ID_PUBLISH', 'mean'),
                               ('DAYS_REGISTRATION', 'mean'),
                               
                               ('EXT_SOURCE_2', 'mean'),
                               ('EXT_SOURCE_3', 'mean')]),
    ]
    
    # Groupby categorical features, calculate the mean and or max 
    # of various numerical statistics.
    for groupby_cols, specs in aggregation_recipes:
        group_object = X.groupby(groupby_cols)
        for select, agg in specs:
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg.upper(), select)
            X = X.merge(group_object[select]
                                  .agg(agg)
                                  .reset_index()
                                  .rename(index=str,
                                          columns={select: groupby_aggregate_name})
                                  [groupby_cols + [groupby_aggregate_name]],
                                  on=groupby_cols,
                                  how='left')
            
    # Get the difference and absolute difference between two 
    # categorical features' mean and or max values of various
    # numerical statistics.
    for groupby_cols, specs in aggregation_recipes:
        for select, agg in specs:
            if agg in ['mean','median','max','min']:
                groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg.upper(), select)
                diff_name = '{}_DIFF'.format(groupby_aggregate_name)
                abs_diff_name = '{}_ABS_DIFF'.format(groupby_aggregate_name)

                X[diff_name] = X[select] - X[groupby_aggregate_name] 
                X[abs_diff_name] = np.abs(X[select] - X[groupby_aggregate_name]) 
            

    # Categorical features
    cat_feat = [
        'NAME_CONTRACT_TYPE', 
        'CODE_GENDER', 
        'FLAG_OWN_REALTY', 
        'NAME_INCOME_TYPE', 
        'NAME_EDUCATION_TYPE', 
        'NAME_FAMILY_STATUS', 
        'NAME_HOUSING_TYPE', 
        'REGION_RATING_CLIENT',
        'REGION_RATING_CLIENT_W_CITY', 
        'WEEKDAY_APPR_PROCESS_START', 
        'ORGANIZATION_TYPE',
        'NAME_TYPE_SUITE', 
        'OCCUPATION_TYPE', 
        'WALLSMATERIAL_MODE', 
        'OBS_30_CNT_SOCIAL_CIRCLE',
        'DEF_30_CNT_SOCIAL_CIRCLE',
        'OBS_60_CNT_SOCIAL_CIRCLE',
        'DEF_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY',
        'AMT_REQ_CREDIT_BUREAU_HOUR',
        'AMT_REQ_CREDIT_BUREAU_WEEK',
        'AMT_REQ_CREDIT_BUREAU_MON',
        'AMT_REQ_CREDIT_BUREAU_QRT',
        'AMT_REQ_CREDIT_BUREAU_YEAR',
        'CNT_CHILDREN',
        'CNT_FAM_MEMBERS',
   
    ]

    # Make copies of each multi-categorical feature, and set aside 
    # those copies to be target mean encoded. 
    #
    # (Some of the original categorical features will later be 
    #  used as numerical features.)
    for feature in cat_feat:
        new_name = feature + '_CAT'
        orig_idx_of_feature = X.columns.get_loc(feature)
        X.insert(orig_idx_of_feature, new_name, X[feature])
        X[[new_name]] = X[[new_name]].apply(lambda x: x.astype('category'))

    # Set aside the categorical features that will also be used as 
    # numerical features.
    cat_feat_to_keep = [
        'CNT_CHILDREN',
        'CNT_FAM_MEMBERS',
        'OBS_30_CNT_SOCIAL_CIRCLE',
        'DEF_30_CNT_SOCIAL_CIRCLE',
        'OBS_60_CNT_SOCIAL_CIRCLE',
        'DEF_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY',
        'AMT_REQ_CREDIT_BUREAU_HOUR',
        'AMT_REQ_CREDIT_BUREAU_WEEK',
        'AMT_REQ_CREDIT_BUREAU_MON',
        'AMT_REQ_CREDIT_BUREAU_QRT',
        'AMT_REQ_CREDIT_BUREAU_YEAR',
    ]


    # Drop the original categorical features that won't also be used as 
    # numerical features:
    cat_feat_to_drop = list(set(cat_feat) - set(cat_feat_to_keep))
    X.drop(cat_feat_to_drop, axis=1, inplace=True)
    
    
    # Engineer some simple binary features:
    
    # 'HAS_CHILDREN': a binary feature that indicates whether or 
    # not a borrower has one or more children.
    X['HAS_CHILDREN'] = X['CNT_CHILDREN'].map(lambda x: 1 if x > 0 else 0)

    # 'HAS_JOB': a binary feature that indicates whether or not 
    # a borrower was employed when their application was submitted.
    X['HAS_JOB'] = X['DAYS_EMPLOYED'].map(lambda x: 1 if x < 0 else 0)
    
    
    # Engineer numerical features:
    
    # Sums
    X['SUM_AMT_INCOME_TOTAL_AMT_ANNUITY'] = X['AMT_INCOME_TOTAL'] + X['AMT_ANNUITY']
    X['TOTAL_ENQUIRIES_CREDIT_BUREAU'] = X[['AMT_REQ_CREDIT_BUREAU_DAY',
                                          'AMT_REQ_CREDIT_BUREAU_HOUR',
                                          'AMT_REQ_CREDIT_BUREAU_WEEK',
                                          'AMT_REQ_CREDIT_BUREAU_MON',
                                          'AMT_REQ_CREDIT_BUREAU_QRT',
                                          'AMT_REQ_CREDIT_BUREAU_YEAR']].sum(axis=1)
    
    # Differences                                                             
    X['DIFF_AMT_CREDIT_AMT_GOODS_PRICE'] = X['AMT_CREDIT'] - X['AMT_GOODS_PRICE']
    X['DIFF_AMT_ANNUITY_AMT_GOODS_PRICE'] = X['AMT_ANNUITY'] - X['AMT_GOODS_PRICE']
    X['DIFF_AMT_INCOME_TOTAL_AMT_ANNUITY'] = X['AMT_INCOME_TOTAL'] - X['AMT_ANNUITY']
    X['CNT_ADULT_FAM_MEMBER'] = X['CNT_FAM_MEMBERS'] - X['CNT_CHILDREN']
    X['DIFF_OBS_30_CNT_SOCIAL_CIRCLE_OBS_60_CNT_SOCIAL_CIRCLE'] = X['OBS_30_CNT_SOCIAL_CIRCLE'] - X['OBS_60_CNT_SOCIAL_CIRCLE']
    X['DIFF_DEF_30_CNT_SOCIAL_CIRCLE_DEF_60_CNT_SOCIAL_CIRCLE'] = X['DEF_30_CNT_SOCIAL_CIRCLE'] - X['DEF_60_CNT_SOCIAL_CIRCLE']

    # Ratios
    X['RATIO_AMT_CREDIT_TO_AMT_ANNUITY'] = X['AMT_CREDIT'] / X['AMT_ANNUITY']
    X['RATIO_AMT_CREDIT_TO_CNT_ADULT_FAM_MEMBER'] = X['AMT_CREDIT'] / X['CNT_ADULT_FAM_MEMBER']
    X['RATIO_AMT_INCOME_TOTAL_TO_AMT_ANNUITY'] = X['AMT_INCOME_TOTAL'] / X['AMT_ANNUITY']
    X['AMT_INCOME_TOTAL_PER_ADULT_FAM_MEMBER'] = X['AMT_INCOME_TOTAL'] / X['CNT_ADULT_FAM_MEMBER']
    X['RATIO_AMT_GOODS_PRICE_TO_LIVINGAREA_AVG'] = X['AMT_GOODS_PRICE'] / X['LIVINGAREA_AVG']
    X['RATIO_AMT_GOODS_PRICE_TO_FLOORSMAX_AVG_AVG'] = X['AMT_GOODS_PRICE'] / X['FLOORSMAX_AVG']
    X['RATIO_AMT_GOODS_PRICE_TO_DAYS_EMPLOYED'] = X['AMT_GOODS_PRICE'] / X['DAYS_EMPLOYED']
    X['RATIO_AMT_GOODS_PRICE_TO_CNT_CHILDREN'] = X['AMT_GOODS_PRICE'] / X['CNT_CHILDREN']
    X['RATIO_AMT_GOODS_PRICE_TO_SUM_AMT_INCOME_TOTAL_AMT_ANNUITY'] = X['AMT_GOODS_PRICE'] / X['SUM_AMT_INCOME_TOTAL_AMT_ANNUITY']
    X['RATIO_AMT_ANNUITY_TO_LIVINGAREA_AVG'] = X['AMT_ANNUITY'] / X['LIVINGAREA_AVG']
    X['RATIO_AMT_ANNUITY_TO_DAYS_EMPLOYED'] = X['AMT_ANNUITY'] / X['DAYS_EMPLOYED']
    X['RATIO_AMT_ANNUITY_TO_CNT_CHILDREN'] = X['AMT_ANNUITY'] / X['CNT_CHILDREN']
    X['RATIO_AMT_ANNUITY_TO_CNT_ADULT_FAM_MEMBER'] = X['AMT_ANNUITY'] / X['CNT_ADULT_FAM_MEMBER']
    X['RATIO_EXT_SOURCE_3_TO_REGION_POPULATION_RELATIVE'] = X['EXT_SOURCE_3'] / X['REGION_POPULATION_RELATIVE']
    X['RATIO_DAYS_LAST_PHONE_CHANGE_TO_DAYS_REGISTRATION'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_REGISTRATION']
    X['PCTG_FAM_CHILDREN'] = X['CNT_CHILDREN'] / X['CNT_FAM_MEMBERS']
    X['PCTG_ENQUIRIES_HOUR'] = X['AMT_REQ_CREDIT_BUREAU_HOUR'] / X['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    X['PCTG_ENQUIRIES_DAY'] = X['AMT_REQ_CREDIT_BUREAU_DAY'] / X['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    X['PCTG_ENQUIRIES_WEEK'] = X['AMT_REQ_CREDIT_BUREAU_WEEK'] / X['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    X['PCTG_ENQUIRIES_MON'] = X['AMT_REQ_CREDIT_BUREAU_MON'] / X['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    X['PCTG_ENQUIRIES_QRT'] = X['AMT_REQ_CREDIT_BUREAU_QRT'] / X['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    X['PCTG_ENQUIRIES_YEAR'] = X['AMT_REQ_CREDIT_BUREAU_YEAR'] / X['TOTAL_ENQUIRIES_CREDIT_BUREAU']

    
    # The idea to make the following features comes 
    # from neptune-ml's solution:
    # https://github.com/neptune-ml/open-solution-home-credit/blob/solution-5/notebooks/eda-application.ipynb
    
    # Based on EXT_SOURCES features
    
    
    # Ratios
    X['RATIO_AMT_CREDIT_TO_AMT_GOODS_PRICE'] = X['AMT_CREDIT'] / X['AMT_GOODS_PRICE']
    X['RATIO_AMT_CREDIT_TO_AMT_INCOME_TOTAL'] = X['AMT_CREDIT'] / X['AMT_INCOME_TOTAL']
    X['RATIO_AMT_CREDIT_TO_CNT_FAM_MEMBERS'] = X['AMT_CREDIT'] / X['CNT_FAM_MEMBERS']
    X['RATIO_AMT_CREDIT_TO_CNT_CHILDREN'] = X['AMT_CREDIT'] / (1 + X['CNT_CHILDREN'])
    X['RATIO_AMT_INCOME_TOTAL_TO_AMT_CREDIT'] = X['AMT_INCOME_TOTAL'] / X['AMT_CREDIT']
    X['RATIO_AMT_INCOME_TOTAL_TO_CNT_CHILDREN'] = X['AMT_INCOME_TOTAL'] / (1 + X['CNT_CHILDREN'])
    X['RATIO_AMT_ANNUITY_TO_AMT_INCOME_TOTAL'] = X['AMT_ANNUITY'] / X['AMT_INCOME_TOTAL']
    X['RATIO_AMT_ANNUITY_AMT_CREDIT'] = X['AMT_ANNUITY'] / X['AMT_CREDIT']
    X['RATIO_CHILDREN_TO_ADULTS'] = X['CNT_CHILDREN'] / X['CNT_ADULT_FAM_MEMBER']
    X['RATIO_DAYS_LAST_PHONE_CHANGE_TO_DAYS_BIRTH'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_BIRTH']
    X['RATIO_DAYS_LAST_PHONE_CHANGE_TO_DAYS_EMPLOYED'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_EMPLOYED']
    X['PCTG_DAYS_EMPLOYED'] = X['DAYS_EMPLOYED'] / X['DAYS_BIRTH']
    
    # Binary categorical features
    X['LONG_EMPLOYMENT'] = (X['DAYS_EMPLOYED'] < -2000).astype(int)
    X['RETIREMENT_AGE'] = (X['DAYS_BIRTH'] < -14000).astype(int)


    # The idea to create these features comes from Olivier's kernel:
    # https://www.kaggle.com/ogrellier/home-credit-hyperopt-optimization
    #X['EXT_SOURCES_PROD'] = X['EXT_SOURCE_1'] * X['EXT_SOURCE_2'] * X['EXT_SOURCE_3']
    X['RATIO_AMT_ANNUITY_TO_AMT_INCOME_TOTAL'] = X['AMT_ANNUITY'] / (1 + X['AMT_INCOME_TOTAL'])
    X['RATIO_AMT_CREDIT_TO_AMT_GOODS_PRICE'] = X['AMT_CREDIT'] / X['AMT_GOODS_PRICE']

    return X

In [15]:
df1=engineer_main_features(df)

In [16]:
df1.shape

(307511, 234)

In [27]:
import lightgbm as lgb


In [77]:
model = lgb.LGBMRegressor(learning_rate=0.09,max_depth=-5,random_state=42,max_iterations=120)
model.fit(X_train,y_train)

[LightGBM] [Warning] Unknown parameter: max_iterations


LGBMRegressor(learning_rate=0.09, max_depth=-5, max_iterations=120,
              random_state=42)

In [78]:
preds = model.predict(X_test)

In [79]:
print(roc_auc_score(y_test,preds))


0.894718359646105


In [52]:
preds[7]

0.0

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [75]:
y=df['TARGET']
x=df.drop(['TARGET','SK_ID_CURR'],axis=1)

In [99]:
cnt=0
for i in y:
    if i==1:
        cnt+=1
print(10000/cnt)

0.4028197381671702


In [98]:
cnt=0
for i in y:
    if i==0:
        cnt+=1
print(10000/cnt)

0.03537493897823026


In [76]:
scaler = StandardScaler()
#features = scaler.fit_transform(df1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state=42) 


In [119]:
lr = LogisticRegression(random_state=0,class_weight={0: 1/92, 1: 1/8})
model = lr.fit(X_train, y_train)
preds = model.predict(X_test)

In [120]:
cnt=0
for i in preds:
    if i==1:
        cnt+=1
print(cnt)

39070


In [40]:
from sklearn.metrics import roc_auc_score
